In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open("names.txt", "r").read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [7]:
block_size = 3
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [8]:
X.shape

torch.Size([32, 3])

In [9]:
C = torch.randn((27, 2))
C

tensor([[ 1.3420,  0.4031],
        [ 0.6655,  2.7057],
        [ 1.3447, -1.3138],
        [-0.0641,  1.3269],
        [ 1.1368, -0.6846],
        [-0.1272, -1.5952],
        [ 1.1861, -1.2079],
        [ 0.5384,  1.6124],
        [-0.8387, -0.8642],
        [ 1.3987,  0.2949],
        [ 1.3452,  0.4731],
        [ 0.3514, -0.0360],
        [ 1.4677,  0.3236],
        [-0.8759, -0.1539],
        [ 0.1562,  1.0837],
        [-0.8438,  2.0374],
        [ 1.0311, -1.2017],
        [-0.7340,  0.3449],
        [ 0.4492, -1.1342],
        [-0.4229, -1.6601],
        [-0.5459,  0.3391],
        [ 1.9489,  0.2915],
        [-1.0009, -1.3991],
        [-0.4897,  1.3142],
        [-1.0489, -0.3961],
        [ 0.8348, -1.0438],
        [-1.6390, -0.3367]])

In [10]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [11]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [12]:
torch.cat(torch.unbind(emb, 1), 1).shape

torch.Size([32, 6])

In [13]:
emb.view(32, 6)

tensor([[ 1.3420,  0.4031,  1.3420,  0.4031,  1.3420,  0.4031],
        [ 1.3420,  0.4031,  1.3420,  0.4031, -0.1272, -1.5952],
        [ 1.3420,  0.4031, -0.1272, -1.5952, -0.8759, -0.1539],
        [-0.1272, -1.5952, -0.8759, -0.1539, -0.8759, -0.1539],
        [-0.8759, -0.1539, -0.8759, -0.1539,  0.6655,  2.7057],
        [ 1.3420,  0.4031,  1.3420,  0.4031,  1.3420,  0.4031],
        [ 1.3420,  0.4031,  1.3420,  0.4031, -0.8438,  2.0374],
        [ 1.3420,  0.4031, -0.8438,  2.0374,  1.4677,  0.3236],
        [-0.8438,  2.0374,  1.4677,  0.3236,  1.3987,  0.2949],
        [ 1.4677,  0.3236,  1.3987,  0.2949, -1.0009, -1.3991],
        [ 1.3987,  0.2949, -1.0009, -1.3991,  1.3987,  0.2949],
        [-1.0009, -1.3991,  1.3987,  0.2949,  0.6655,  2.7057],
        [ 1.3420,  0.4031,  1.3420,  0.4031,  1.3420,  0.4031],
        [ 1.3420,  0.4031,  1.3420,  0.4031,  0.6655,  2.7057],
        [ 1.3420,  0.4031,  0.6655,  2.7057, -1.0009, -1.3991],
        [ 0.6655,  2.7057, -1.0009, -1.3

In [17]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[ 0.9435, -0.7417, -0.5058,  ..., -0.2477,  0.9073, -0.9885],
        [ 0.0810, -0.9978, -0.9205,  ..., -0.8832,  0.9782, -0.2662],
        [ 0.5653, -0.9962, -0.9810,  ..., -1.0000,  0.9901, -0.9418],
        ...,
        [-1.0000, -0.2878, -0.9184,  ..., -0.7737,  0.9999, -0.9996],
        [ 0.9972, -0.9489, -0.0189,  ..., -0.9991, -0.7404,  0.7695],
        [ 0.9986,  0.9861,  0.9957,  ...,  1.0000, -1.0000, -0.8841]])

In [18]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [19]:
logits = h @ W2 + b2
logits.shape

torch.Size([32, 27])

In [20]:
counts = logits.exp()

In [21]:
prob = counts / counts.sum(1, keepdims=True)

In [22]:
prob.shape

torch.Size([32, 27])

In [24]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(16.9542)